In [ ]:
!pip install pycolmap

In [2]:
import pycolmap

In [3]:
imgs = "/path/to/imgs"
db = "/path/to/database/db.db"
output_path = '/path/to/output/folder'

### Feature extraction

In [22]:
# If all images come from the same sensor, use 'SINGLE'. It will give a significant boost to accuracy
# because of a better estimation of intrisics
camera_mode = 'SINGLE' # 'SINGLE' or 'PER_FOLDER' or 'PER_IMAGE'

# If few images share the same intrinsics, use a camera model with less parameters
camera_model = 'SIMPLE_RADIAL' # 'SIMPLE_PINHOLE', 'PINHOLE', 'SIMPLE_RADIAL', 'RADIAL', 'OPENCV', 'FULL_OPENCV', 'SIMPLE_RADIAL_FISHEYE', 'RADIAL_FISHEYE', 'OPENCV_FISHEYE', 'FOV', 'THIN_PRISM_FISHEYE'

sift_opt = pycolmap.SiftExtractionOptions()
sift_opt.max_image_size = 1024 # Extract features at low resolution could significantly reduce the overall accuracy
sift_opt.max_num_features = 2048 # Generally more features is better, even if behond a certain number it doesn't help incresing accuracy
sift_opt.upright = True # rotation invariance
device = 'cpu' # 'auto' or 'cpu' or 'cuda'

In [ ]:
print("Extracting RootSIFT features ..")
pycolmap.extract_features(database_path=db,
                          image_path=imgs,
                          camera_mode=camera_mode,
                          camera_model=camera_model,
                          sift_options=sift_opt,
                          device=device,
                          verbose=True)

In [ ]:
# See default options and doc
pycolmap.extract_features.__doc__
help(pycolmap.SiftExtractionOptions)
print(pycolmap.SiftExtractionOptions().summary())

### Matching

In [24]:
matching_opt = pycolmap.SiftMatchingOptions()
matching_opt.max_ratio = 0.80 # Ratio threshold significantly influence the performance of the feature extraction method. It varies depending on the local feature but also on the image type 
matching_opt.max_distance = 0.7
matching_opt.cross_check = True
matching_opt.max_error = 1.0 # The ransac error threshold could help to exclude less accurate tie points
device = 'cpu'

In [ ]:
print("Exhaustive matching ...")
pycolmap.match_exhaustive(db, sift_options=matching_opt, device=device)

In [ ]:
# See default options and doc
pycolmap.match_exhaustive.__doc__
print(pycolmap.SiftMatchingOptions().summary())
help(pycolmap.SiftMatchingOptions)

### Incremental reconstruction

In [10]:
mapper_options = pycolmap.IncrementalMapperOptions()
mapper_options.extract_colors = False
mapper_options.min_model_size = 3

# Sometimes you want to impose the first image pair for initialize the incremental reconstruction
mapper_options.init_image_id1 = -1 
mapper_options.init_image_id2 = -1

# Choose which interior will be refined during BA
mapper_options.ba_refine_focal_length = True
mapper_options.ba_refine_principal_point = True
mapper_options.ba_refine_extra_params = True

In [11]:
maps = pycolmap.incremental_mapping(database_path=db, image_path=imgs, output_path=output_path, options=mapper_options)

In [ ]:
# See default options and doc
pycolmap.incremental_mapping.__doc__
help(pycolmap.IncrementalMapperOptions)
print(pycolmap.IncrementalMapperOptions().summary())

In [14]:
reconstruction = pycolmap.Reconstruction(f"{output_path}/0")
reconstruction.write_text(f"{output_path}/0")

In [ ]:
# Overview pycolmap functions
dir(pycolmap)